In [22]:
rho = 0.96287 # kg/m^3
V = 102.88 # m/s
q = 0.5*rho*V**2 # N/m^2

print(q)

rho = 0.00186850 # slugs/cu ft, at 8000 ft
V = 337.562 # ft/s, 200 knots
q = 0.5*rho*V**2 # slugs/sq ft dynamic pressure at cruise

print(q*32.2)

print(q)

5095.649774463999
3427.883715723476
106.45601601625701


# Static Stability

This section demonstrates longitudinal stability analysis using example airplane. Specifically, neutral point and static margin - two important parmeters - are estimated. The neutral point denotes a CG position at which the aircraft will be neutrally stable. This implies any movement of CG beyond this point leads to aircraft being unstable. Refer to lecture notes and Section 16.3.1 and 16.3.2 in Raymer for more details.

Using equation 16.9 in Raymer, neutral point can be estimated using:

<!-- $$
    \bar{X}_{np} = \frac{ C_{L_\alpha} \bar{X}_{ac_w} - C_{M_{\alpha_f}} + \eta_h \dfrac{S_h}{S_w} C_{L_{\alpha_h}} \dfrac{\partial \alpha_h}{\partial \alpha} \bar{X}_{ac_h} + \dfrac{F_{p_\alpha}}{qS_w} \dfrac{\partial \alpha_p}{\partial \alpha} \bar{X}_p } { C_{L_\alpha} + \eta_h \dfrac{S_h}{S_w} C_{L_{\alpha_h}} \dfrac{\partial \alpha_h}{\partial \alpha} + \dfrac{F_{p_\alpha}}{qS_w} \dfrac{\partial \alpha_p}{\partial \alpha} }
$$ -->

$$
    \bar{X}_{np} = \frac{ C_{L_\alpha} \bar{X}_{ac_w} - C_{M_{\alpha_f}} + \eta_h \dfrac{S_h}{S_w} C_{L_{\alpha_h}} \dfrac{\partial \alpha_h}{\partial \alpha} \bar{X}_{ac_h} } { C_{L_\alpha} + \eta_h \dfrac{S_h}{S_w} C_{L_{\alpha_h}} \dfrac{\partial \alpha_h}{\partial \alpha} }
$$

Each term in the above equation represents pitching moment contribution from different components. The first term represents contribution from wing pitching moment. 

Note that $\bar{X}$ in above equation represents distance normalized with mean aerodynamic chord.



> __*NOTE*__: In the above equation, effect of thrust/propulsion system are ignored. Based on the recommendation of 

First term is a contribution due to wing pitching moment $C_{L_\alpha}$ is lift curve slope for wing (known) and $\bar{X}_{ac_w}$ is already known

Each term in this equation should be adjusted for the flap deflection (when doing takeoff and landing trim analysis)

Below sections compute 

## Wing

The lift model for the wing can be written as

$$
    C_{L_w} = C_{L_{\alpha_w}} (\alpha + i_w - \alpha_{0_{L_w}}),
$$

where $C_{L_{\alpha_w}}$ is the lift curve slope for the wing, $i_w$ is the incidence angle, $\alpha_{0_{L_w}}$ is the zero-lift angle of attack for the wing. The $C_{L_{\alpha_w}}$ and $alpha_{0_{L_w}}$ are already computed in aerodynamics section. The $i_w = 2^{\circ}$ is determined in wing planform section. Note that $i_w$ can be further tuned if required.

## Tail

Lift model for the tail is written as

$$
    C_{L_h} = C_{L_{\alpha_h}} ( \alpha + i_h - \epsilon - \alpha_{0_{L_h}} )
$$

where $C_{L_{\alpha_h}}$ is the horizontal lift curve slope, $i_h$ is the tail incidence angle, and $\epsilon$ refers to downwash.

The $C_{L_{\alpha_h}}$ is computed using the same method as described in [aerodynamics section](../aerodynamics/lift_curve.ipynb).

## Fuselage Pitching moment

$C_{M_{\alpha_f}}$ is the pitching moment contribution from fuselage, can be computed using Section 16.3.8 in Raymer

Position of root quarter chord as percent of fuselage length = 0.37 (from position of components in CG section)

In [ ]:
Kfus = 0.014 # from figure 16.14
Wf = 5.0 # ft
Lf = 31.6 # ft
mac = 4.3 # ft
Sw = 134 # sq ft

Cm_alpha_fuselage = Kfus * Wf**2 * Lf / mac / Sw

print(f"Fuselage pitching moment: {Cm_alpha_fuselage:.4f}") # 1/deg

Fuselage pitching moment: 0.0192


## Thrust Effects

Section 16.3.9 Raymer

$$
    F_{P_\alpha} = qN_BA_p \frac{\partial C_{N_{blade}}}{\partial \alpha} f(T)
$$

$N_B$ = number of blades = 3 

$A_P$ = area of the propeller disk 



In [ ]:
import numpy as np

rho = 0.00186850 # slugs/cu ft, at 8000 ft
V = 337.562 # ft/s, 200 knots
q = 0.5*rho*V**2 # lbs/sq ft dynamic pressure at cruise

n = 0.8*2700/60 # rps
D = 5.8 # ft
NB = 3
Ap = np.pi*D**2/4

J = V/n/D

print(J)

dCNblade_dalpha = 0.055 # 1/rad, figure 16.15

K = 1/np.pi/8/0.8

CD0 = 0.03363

W = 0.95*5374

CL = W/0.5/rho/V**2

CD = CD0 + K*CL**2

D = 0.5*rho*V**2 * CD

print(32.2*D/rho/V**2/Sw**2)

fT = 1

Fp_alpha = q*NB*Ap*dCNblade_dalpha*fT

print(Fp_alpha*np.pi/180/32.2)

print()

1.616676245210728
0.10259264220444914
0.25154837597261187


In [ ]:
de_da = 0.05

dap_da = 1 + de_da